# Detector de idiomas

Se va a diseñar un detector de idiomas utilizando recursos de **Machine Learning**. 
La idea principal es centrarlo en un problema de clasificación, con el que vamos a ayudarnos de una potente herramienta de ML para Python, **Scikit-Learn**.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from time import time
from sklearn import cross_validation
from pprint import pprint
from sklearn import ensemble
from sklearn import feature_extraction
from sklearn import linear_model
from sklearn import pipeline
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcess
from sklearn.grid_search import GridSearchCV
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Vamos a trabajar con corpus del Parlamento Europeo que se encuentran disponibles en el siguiente enlace:
http://www.statmt.org/europarl/

Abrimos los diferentes corpus.
Aclaramos los idiomas:

pl = polaco  
it = italiano  
es = español  
de = alemán  
en = inglés  
hu = húngaro  
pt = portugués  

In [2]:
pl = open('corpus/pl','r', encoding='utf8')
it = open('corpus/it','r', encoding='utf8')
es = open('corpus/es','r', encoding='utf8')
de = open('corpus/de','r', encoding='utf8')
en = open('corpus/en','r', encoding='utf8')
hu = open('corpus/hu','r', encoding='utf8')
pt = open('corpus/pt','r', encoding='utf8')

In [3]:
corpus_list = ['pl','it','es','de','en','hu','pt']

Con la variable *size* le vamos a decir al programa la cantidad de frases con las que vamos a trabajar. Para que sea más fácil y todos los idiomas estén igualmente representados, procuraremos que *size* sea múltimo de 7

In [4]:
size = 4000*7

Vemos que solo hemos introducido veintiocho mil frases, o lo que es lo mismo, cuatro mil frases de cada idioma. Hemos tenido que coger un número tan reducido de frases debido a problemas de memoria física, ya que consume muchos recursos resolver este problema con el método que describiremos más adelante, aunque también podrá verse que conseguimos unos resultados muy buenos pese a entrenar con pocas frases en comparación con todas las que tienen los corpus.

A continuación vamos a crear el DataFrame que estará constituido simplemente con dos columnas, la primera será una frase y la segunda el idioma de dicha frase.

In [5]:
t0 = time()

leng = []
txt = []

for j in corpus_list:
    n = 0
    for i in eval(j):
        if n < (size/len(corpus_list)):
            if len(i) < 40:
                pass
            else:
                n+=1
                txt.append(i[:-1])
                leng.append(str(j))
        else:
             break
    eval(j).close()
                
df = pd.DataFrame()
df['texto'] = txt
df['idioma'] = leng

print("DataFrame creado en %0.4f s" % (time() - t0))

DataFrame creado en 0.0419 s


Podemos ver un pequeño resumen de nuestro DataFrame

In [6]:
df.describe()

,texto,idioma
count,28000,28000
unique,27189,7
top,Az előző ülés jegyzőkönyvének elfogadása: lásd...,en
freq,46,4000


Eliminamos las frases repetidas, ya que no son interesantes para este problema.

In [7]:
df.drop_duplicates(subset='texto', keep='first', inplace=True)

In [8]:
df.describe()

,texto,idioma
count,27189,27189
unique,27189,7
top,"Signor Presidente, signor Commissario, onorevo...",pt
freq,1,3994


Crearemos, a partir del DataFrame, nuestro conjunto de entrenamiento, train, y nuestro conjunto de validación, test.
Para realizar esto utilizaremos la herramienta de Scikit-Learn *train_test_split* cuyo parámetro *test_size* nos va a indicar el tamaño del conjunto de validación.

In [9]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    df['texto'],
    df['idioma'],
    test_size=0.3,
    random_state=0
)

Con la funcion *feature_extraction.text.TfidfVectorizer* podremos separar la frase por palabras, lo que se conoce como *tokenizar*. El parámetro *analyzer='word'* nos indica que vamos a separar por palabras.

In [10]:
vectorizer = feature_extraction.text.TfidfVectorizer(
    analyzer='word',
    ngram_range = (1,2),
)

Se va a entrenar diferentes clasificadores para su evaluación con la métrica *accuracy* y nos quedaremos con el que más acierto presente.

In [11]:
classifiers = [XGBClassifier(objective='multi:softmax', n_jobs=-1),
               RandomForestClassifier(n_estimators=200, max_depth=10, n_jobs=-1),
               KNeighborsClassifier(n_neighbors=7, n_jobs=-1),
               AdaBoostClassifier(n_estimators=200, learning_rate=0.5)
              ]

for clf in classifiers:
    model = pipeline.Pipeline([
        ('vectorizer', vectorizer),
        ('model', clf)
    ])
    print('Entrenando modelo {}'.format(clf))
    model.fit(X_train, y_train)
    print('Acierto en redicciones:\n')
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(acc)

Entrenando modelo XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='multi:softmax', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
Acierto en redicciones:



/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9680029422581832
Entrenando modelo RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)
Acierto en redicciones:

0.9849209268113277
Entrenando modelo KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=7, p=2,
           weights='uniform')
Acierto en redicciones:

0.995341424543337
Entrenando modelo AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.5, n_estimators=200, random_state=None)
Acierto en redicciones:

0.928282456785583


Ahora volvemos a definir nuestro mejor modelo, en este caso KNeighborsClasifier.

Por supuesto, se podrían mejorar los otros modelos haciendo un *tuning* de parametros por validación cruzada

In [12]:
vectorizer = feature_extraction.text.TfidfVectorizer(
    analyzer='word'
)

model = pipeline.Pipeline([
    ('vectorizer', vectorizer),
    ('kNN', KNeighborsClassifier(n_neighbors=7, n_jobs=-1))
])

Ya podemos entrenar nuestro modelo. Con la función *fit* entrenamos con el conjunto train, y con la función *predict* hacemos las predicciones en el conjunto test.

In [13]:
t1 = time()

print("Entrenando el modelo")
model.fit(X_train, y_train)
print("Entrenado")

print("Haciendo predicciones")
y_predicted = model.predict(X_test)
print("Predicciones realizadas")

print("Tiempo de entrenamiento y testeo: %0.4f s " % (time() - t1) +
      "con %0.0f frases" % (len(df)))

cm = metrics.confusion_matrix(y_test, y_predicted, labels=corpus_list)

Entrenando el modelo
Entrenado
Haciendo predicciones
Predicciones realizadas
Tiempo de entrenamiento y testeo: 6.2862 s con 27189 frases


Podemos visualizar una matriz de confusión para hacer una primera comprobación tanto de los errores como de los aciertos cometidos por el modelo

In [14]:
pd.DataFrame(cm,index=corpus_list,columns=corpus_list)

,pl,it,es,de,en,hu,pt
pl,1093,0,2,1,3,0,0
it,2,1199,1,0,2,0,0
es,0,0,1160,0,1,0,4
de,4,0,0,1197,0,0,0
en,0,0,0,0,1219,0,0
hu,5,0,0,0,1,1040,1
pt,0,1,11,1,0,0,1209


Podemos ver el error cometido teniendo en cuenta que los elemento de la diagonal principal son aciertos y el resto errores. Así que para evaluar nuestro modelo, sumaremos los aciertos y lo dividiremos por el número total de frases que tenemos. Ésto nos dará el porcentaje de acierto total.

In [15]:
sum_diag = sum(np.einsum('ii -> i', cm))
acc = (sum_diag/len(X_test))
acc

0.9950962363614074

También podemos ver, a través de la función *classification_report* la precisión del modelo por cada idioma.

In [16]:
print(metrics.classification_report(y_test, y_predicted,
                                    target_names=corpus_list))

             precision    recall  f1-score   support

         pl       1.00      1.00      1.00      1201
         it       0.99      1.00      1.00      1219
         es       0.99      1.00      0.99      1165
         de       1.00      0.99      1.00      1047
         en       1.00      1.00      1.00      1204
         hu       0.99      0.99      0.99      1099
         pt       1.00      0.99      0.99      1222

avg / total       1.00      1.00      1.00      8157



Ahora podemos guardar nuestras predicciones en un dataframe.

In [17]:
predictions = pd.DataFrame()
predictions['texto'] = X_test
predictions['idioma'] = y_predicted
predictions[0:10]

,texto,idioma
6350,Si tratta ora di rimuovere al più presto il le...,it
22768,Ez nyilvánvalóan a jogszabály hiányossága.,hu
9973,Y los controles no deben consistir en que el i...,es
13900,Es gibt ja bekanntlich ein Memorandum von Pari...,de
14638,Sie muß im Blickpunkt der Öffentlichkeit stehen.,de
19392,I also look forward to the White Paper which w...,en
22128,"Nem túl nagy, és talán általában van ok az inf...",hu
21085,Hölgyeim és Uraim! A legutolsó parlamenti ülés...,hu
18887,"Thank you very much, Mr Patten, for your inter...",en
15629,Eine solch weitgehende Taktik der verbrannten ...,de


Como sabemos que en los corpus hay frases de idiomas que no corresponden al corpus donde está, podemos hacer predicciones con nuestro modelo sobre estas frases para intentar corregir los errores que hay.

In [18]:
x_predicted_total = model.predict(df['texto'])

Ahora podemos crear un nuevo dataframe con las frases de entrenamiento y la prediccion de los idiomas, para posteriormente poder hacer unas nuevas predicciones con este nuevo conjunto de entrenamiento corregido y ver si hemos conseguido mejorar el modelo o no.

In [19]:
df_corregido = pd.DataFrame()
df_corregido['texto'] = df['texto']
df_corregido['idioma'] = x_predicted_total

In [20]:
X_train_new, X_test_new, y_train_new, y_test_new = cross_validation.train_test_split(
    df_corregido['texto'],
    df_corregido['idioma'],
    test_size=0.3,
    random_state=0
)

In [21]:
vectorizer = feature_extraction.text.TfidfVectorizer(
    ngram_range=(0,1),
    analyzer='word'
)

new_model = pipeline.Pipeline([
    ('vectorizer', vectorizer),
    ('kNN', KNeighborsClassifier(n_neighbors=7, n_jobs=-1))
])

t1 = time()

print("Entrenando el modelo")
new_model.fit(X_train_new, y_train_new)
print("Entrenado")

print("Haciendo predicciones")
y_predicted_new = new_model.predict(X_test_new)
print("Predicciones realizadas")

print("Tiempo de entrenamiento y testeo: %0.4f s " % (time() - t1) +
      "con %0.0f frases" % (len(df_corregido)))

cm_new = metrics.confusion_matrix(y_test_new, y_predicted_new, labels=corpus_list)

Entrenando el modelo
Entrenado
Haciendo predicciones
Predicciones realizadas
Tiempo de entrenamiento y testeo: 6.2221 s con 27189 frases


In [22]:
pd.DataFrame(cm_new,index=corpus_list,columns=corpus_list)

,pl,it,es,de,en,hu,pt
pl,1103,0,0,0,1,0,0
it,2,1198,0,0,0,0,0
es,0,0,1174,0,0,0,0
de,1,0,0,1197,1,0,0
en,0,0,0,0,1226,0,0
hu,1,0,0,0,0,1039,0
pt,0,0,3,0,0,0,1211


In [23]:
sum_diag_new = sum(np.einsum('ii -> i', cm_new))
acc_new = (sum_diag_new/len(X_test_new))
acc_new

0.9988966531813167

Vemos ahora que el porcentaje de acierto ha aumentado hasta casi el 100%, por lo que hemos conseguido corregir algunos errores con esta técnica de intentar predecir sobre el mismo conjunto de entrenamiento. 

In [24]:
new_predictions = pd.DataFrame()
new_predictions['texto'] = X_test_new
new_predictions['idioma'] = y_predicted_new
new_predictions[0:10]

,texto,idioma
6350,Si tratta ora di rimuovere al più presto il le...,it
22768,Ez nyilvánvalóan a jogszabály hiányossága.,hu
9973,Y los controles no deben consistir en que el i...,es
13900,Es gibt ja bekanntlich ein Memorandum von Pari...,de
14638,Sie muß im Blickpunkt der Öffentlichkeit stehen.,de
19392,I also look forward to the White Paper which w...,en
22128,"Nem túl nagy, és talán általában van ok az inf...",hu
21085,Hölgyeim és Uraim! A legutolsó parlamenti ülés...,hu
18887,"Thank you very much, Mr Patten, for your inter...",en
15629,Eine solch weitgehende Taktik der verbrannten ...,de


A continuación vamos a crear una función sencilla, lo que sería el detector en sí, donde el parámetro de entrada será una frase y la salida será el idioma de dicha frase.

In [25]:
def detection(x):
    lenguage=new_model.predict([x])
    if lenguage == 'es':
        lenguage = 'español'
    elif lenguage == 'pt':
        lenguage = 'portugués'
    elif lenguage == 'it':
        lenguage = 'italiano'
    elif lenguage == 'pl':
        lenguage = 'polaco'
    elif lenguage == 'de':
        lenguage = 'alemán'
    elif lenguage == 'en':
        lenguage = 'inglés'
    elif lenguage == 'hu':
        lenguage = 'húngaro'
        
    print('El idioma de la frase es: '+lenguage)

Evaluemos algunos ejemplos:

In [26]:
detection('hola, mi nombre es Rodrigo')

El idioma de la frase es: español


In [27]:
detection('hello, my name is Rodrigo')

El idioma de la frase es: inglés


In [28]:
detection('Alguem aqui fala espanhol?')

El idioma de la frase es: portugués


In [29]:
detection('Nett, Sie kennen zu lernen')

El idioma de la frase es: alemán


In [30]:
detection('Piacere di conoscerla')

El idioma de la frase es: italiano
